Keras Tuner는 TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트를 선택하는데 도움을 주는 라이브러리이다. 머신러닝 애플리케이션에 대한 올바른 하이퍼파라미터 세트를 선택하는 과정을 하이퍼파라미터조정 또는 하이퍼튜닝 이라고 한다.

하이퍼파라미터는 훈련 프로세스 및 ML 모델의 토폴로지를 제어하는 변수이다. 이러한 변수는 훈련 과정에서 일정하게 유지되며 ML 프로그램의 성능에 직접적으로 영향을 미친다. 하이퍼파라미터에는 두가지 유형이 있다.

1. 숨겨진 레이어의 수 및 너비와 같이 모델 선택에 영향을 미치는 모델 하이퍼파라미터
2. SGD의 학습률 및 KNN 분류자의 최근접 이웃 수와 ㅎ같은 학습 알고리즘의 속도와 품질에 영향을 주는 알고리즘 하이퍼파라미터

In [ ]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt

# 데이터세트 다운로드 준비하기

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

# 모델 저의하기

하이퍼 튜닝을 위한 모델을 빌드할 때는 모델 아키텍처와 더불어 하이퍼 파라미터 검색 공간도 정의한다. 하이퍼 튜닝을 위해 설정하는 모델을 하이퍼 모델이라고 한다.

두가지 접근 방식을 통해 하이퍼 모델을 정의할 수 있다.

> 모델 빌더 함수 사용

> Keras Tuner API의 HyperModel 클래스를 하위 클래스화

또한 두 개의 사전 정의된 HyperModel-클래스인 HyperXception 과 HyperResNet을 컴퓨터 비전 애플리케이션에 사용할 수 있다.


In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

# 튜너를 인스턴스화하고 하이퍼튜닝 수행하기

튜너를 인스턴스화하여 하이퍼튜닝을 수행한다. Keras Tuner에는 RandomSearch, Hyperband, BayesianOptimization 및 Sklearn이 있다. 

Hyperband 튜너를 인스턴스화 하려면 최적화할 하이퍼모델인 objective  및 훈련할 최대 epoch 수를 지정해야 한다.

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

Hyperband 튜닝 알고리즘은 적응형 리소스 할당 및 조기 중단을 사용하여 고성능 모델에서 신속하게 수렴한다. 이것은 스포트 챔피언십 스타일 브래킷을 사용하여 수행된다. 이 알고리즘은 몇 개의 epoch에 대해 많은 수의 모델을 훈련하고 최고 성능을 보이는 절반만 다음 단계로 넘긴다. Hyperband는 1 + log factor(max_epochs)를 계산하고 이를 가장 가까운 정수로 반올림하여 한 브래킷에서 훈련할 모델 수를 결정한다.

하이퍼파라미터 검색을 실행하기 전에 훈련단계가 끝날때마다 훈련 결과를 지우도록 콜백을 정의한다.

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

결론은... Keras Tuner는 모르겠다...